In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import os
from tkinter.filedialog import askopenfilename
import tkinter as tk
from tkinter import Tk
import datetime as dt

In [ ]:
print(os.path.dirname(__file__))

In [2]:
root = Tk()
root.withdraw()
filename = askopenfilename(initialdir = os.getcwd())
# Check root is destroyed
root.destroy()

In [11]:
# print(os.getcwd())

'/Users/maka.ibe/Documents/GitHub/project-portfolio/scraping_jira_tickets'

In [10]:
# print(filename)

'/Users/maka.ibe/Documents/GitHub/project-portfolio/scraping_jira_tickets/01_html/2021_02_17_your_work.htm'

In [3]:
class JiraTickets:
    def __init__(self, file):
        self.file = file
        self._tickets_soup = self._make_soup()
        self._all_tickets = self._get_all_ticket_info()
        self._dict_keys = ['span_class','small_class','title', 'ticket_ref','other','group','status']
        self.tickets_df = self._create_tickets_df()
        self.tickets_count = len(self.tickets_df)
    
    def _make_soup(self):
        return BeautifulSoup(open(self.file), "lxml")
    
    def _get_all_ticket_info(self):
        attr_value_regex = re.compile(r"global-pages.home.ui.tab-container.tab.item-list.item-link#issue-[0-9]{6}")
        all_tickets = self._tickets_soup.find_all(attrs={"data-test-id":attr_value_regex})
        return all_tickets
    
    def _create_ticket_dict(self, x):
    #   Find all span tags  
        data = x.find_all('span')
    #   Flatten list of lists into a single list of string items
        flat_data = [str(item) for sublist in data for item in sublist]
    #   Create dict
        output = dict(zip(self._dict_keys, flat_data))
        return output
    
    def _create_tickets_df(self):    
        dict_output = []
        for x in self._all_tickets:
            dict_output.append(self._create_ticket_dict(x))
        df = pd.DataFrame(dict_output) 
        df['url'] = df.ticket_ref.map('https://justeattakeaway.atlassian.net/browse/{}'.format)
        columns = self._dict_keys.copy()
        columns.append('url')
        return df[columns]
    

In [4]:
my_file = 'jira_your_work.htm'

In [5]:
x = JiraTickets(filename)

In [6]:
# x.file

In [7]:
x.tickets_df.head()

,span_class,small_class,title,ticket_ref,other,group,status,url
0,"<span class=""sc-kjKmiN iWKWoC"">Analysis into t...","<small class=""sc-dwLhYX iVsqEk""><span>PA-1194<...",Analysis into typing time in Zendesk,PA-1194,·,Product Analytics,On Hold (Action),https://justeattakeaway.atlassian.net/browse/P...
1,"<span class=""sc-kjKmiN iWKWoC"">Add re-open rat...","<small class=""sc-dwLhYX iVsqEk""><span>BUS-4340...",Add re-open rate view to async contacts dashboard,BUS-4340,·,Business Intelligence,Reopened,https://justeattakeaway.atlassian.net/browse/B...
2,"<span class=""sc-kjKmiN iWKWoC"">Interim snapsho...","<small class=""sc-dwLhYX iVsqEk""><span>BUS-4284...",Interim snapshot numbers for offline bad items...,BUS-4284,·,Business Intelligence,Pending,https://justeattakeaway.atlassian.net/browse/B...
3,"<span class=""sc-kjKmiN iWKWoC"">Async SLA First...","<small class=""sc-dwLhYX iVsqEk""><span>BUS-4283...",Async SLA First Response Fix and Investigation,BUS-4283,·,Business Intelligence,Work in progress,https://justeattakeaway.atlassian.net/browse/B...
4,"<span class=""sc-kjKmiN iWKWoC"">Ingest data for...","<small class=""sc-dwLhYX iVsqEk""><span>BUS-4197...",Ingest data for Offline Queries - Bad & Incorr...,BUS-4197,·,Business Intelligence,Work in progress,https://justeattakeaway.atlassian.net/browse/B...


In [8]:
print(f"Number of tickets currently open: {x.tickets_count}")

Number of tickets currently open: 15


In [9]:
x.tickets_df.to_csv(os.path.join(os.getcwd()
                                 , '02_csv'
                                 , dt.datetime.today().strftime('%Y_%m_%d_%H%M%S')+'jira_tickets.csv'))